# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [14]:
Summaries_file = 'fungus_Summaries.pkl.bz2'
Abstracts_file = 'fungus_Abstracts.pkl.bz2'

In [15]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for an example of a paper:

In [16]:
Summaries[28425998]

paper(title='Drivers of salamander extirpation mediated by Batrachochytrium salamandrivorans.', authors=['Stegen G', 'Pasmans F', 'Schmidt BR', 'Rouffaer LO', 'Van Praet S', 'Schaub M', 'Canessa S', 'Laudelout A', 'Kinet T', 'Adriaensen C', 'Haesebrouck F', 'Bert W', 'Bossuyt F', 'Martel A'], year=2017, doi='10.1038/nature22059')

In [17]:
Abstracts[28425998]

"The recent arrival of Batrachochytrium salamandrivorans in Europe was followed by rapid expansion of its geographical distribution and host range, confirming the unprecedented threat that this chytrid fungus poses to western Palaearctic amphibians. Mitigating this hazard requires a thorough understanding of the pathogen's disease ecology that is driving the extinction process. Here, we monitored infection, disease and host population dynamics in a Belgian fire salamander (Salamandra salamandra) population for two years immediately after the first signs of infection. We show that arrival of this chytrid is associated with rapid population collapse without any sign of recovery, largely due to lack of increased resistance in the surviving salamanders and a demographic shift that prevents compensation for mortality. The pathogen adopts a dual transmission strategy, with environmentally resistant non-motile spores in addition to the motile spores identified in its sister species B. dendrob

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [18]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [19]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(28425998)
display_summary(28425998, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the titles and abstracts of the papers in our dataset. We will implement our inverted index as a Python dictionary with term strings as keys and posting lists (implemented as Python lists) as values. We include all the tokens we can find in the title and (if available) in the abstract:

In [20]:
from collections import defaultdict

inverted_index = defaultdict(list)

# This can take a few seconds:
for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)
#print(term_set)        

Let's see what's in the index for the example term 'amsterdam':

In [21]:
print(inverted_index['amsterdam'])

[13577999, 22679594, 23155502, 28879757, 30722636]


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'madrid':

In [22]:
query_word = 'madrid'
for i in inverted_index[query_word]:
    display_summary(i, show_abstract=True)

----------

# Tasks

**Your name:** ...

### Task 1

Implement the function `and_merge` outlined below. This function takes two posting lists from the index that can be assumed to be sorted already, and it should return the result of the merging of the two lists with AND. The resulting list should therefore include all the elements that appear in both lists. As explained on the slides, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Then, test your function with an example.

In [10]:
"""
It loops twice, i believe, therefore, the solution below is the correct one.
"""

a = [2,4,6]
b = [1,2,3,4,5]

def and_merge(sorted_list1, sorted_list2):
    merged_list = []
    
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    
    # ---
    for elem in list1: 
        if elem in list1 and elem in list2:
            merged_list.append(elem)
    return merged_list
    # ---


# testing:
print(and_merge(a,b))

[2, 4]


In [11]:


a = [2,4,6]
b = [1,2,3,4,5]

def and_merge(sorted_list1, sorted_list2):
    merged_list = []
    
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    lst = list1+list2

    # ---
    dupes = [x for n, x in enumerate(lst) if x in lst[:n]]
    # ---
    return dupes

# testing:
print(and_merge(a,b))

[2, 4]


### Task 2

Similarly as above, implement the function `or_merge` outlined below that executes an OR merging of the lists. The resulting list should therefore include all the elements that appear in at least one of the lists. Again, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Elements that appear in both input list should only appear once in the output list. Test your function again with an example.

In [12]:
"""
Solution #1 for task2
"""
from collections import OrderedDict
import itertools
b = [1,2,4,5]
a = [1,5,11,23,45,50]
    
def or_merge(sorted_list1, sorted_list2):
    merged_list = []    
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    # ---
    lst = list(set(list1+list2))
    new_lst = []
    while lst:
        minimum = lst[0] #arbitrary number
        for x in lst:
            if x < minimum:
                minimum = x
        new_lst.append(minimum)
        lst.remove(minimum)
    return new_lst
    # ---
    

# testing:
print(or_merge(a,b))

[1, 2, 4, 5, 11, 23, 45, 50]


In [13]:
"""
This one works as well, but it loops twice the elements
"""


a = [1,2,4,5]
b = [1,5,11,23,45,50]

def or_merge(sorted_list1, sorted_list2):
    merged_list = []    
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    # ---
    size_1 = len(list1)
    size_2 = len(list2)
    lst = list(set(list1+list2)) #with the set we remove duplicates
    for i in range(len(lst)):
        for j in range(i+1,len(lst)):
            if lst[i]> lst[j]:
                lst[i],lst[j] = lst[j],lst[i]
    return lst


print(or_merge(a,b))

[1, 2, 4, 5, 11, 23, 45, 50]


### Task 3

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, and returns as function value a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list.

For that, access the variable `inverted_index` from above and use the method `and_merge` that you defined. Also use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query.

Again demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

In [14]:
#retrieving a query with two words!
def and_query(a_query):
    merge_list = []
    elem = preprocess(tokenize(a_query))
    merge_list = inverted_index[elem[0]]
    for word in elem[1:]:
        merge_list = and_merge(merge_list, inverted_index[word])
    
    return merge_list
        
c = 'basidiomycete Heterobasidion annosum'
d = and_query(c)  
for i in d:
        display_summary(i, show_abstract = True)
print(len(d))

13


In [24]:
def remove_duplicates(a_list):
    result = []
    for x in a_list:
        if x not in result:
            result.append(x)
    
    return result

#retrieving a query with two words!
import more_itertools
def and_query(a_query):
    elem = preprocess(tokenize(a_query))
    #print(elem)
    lst = {}
    lst2 = []
    for i in elem:
        #print(i)
        lst[i] = inverted_index[i]
    for i,j in lst.items():
        lst2.append(j)
    #print(len(lst2))
    
    new_lst = []
    for i, element in enumerate(lst2): 
        previous_element = lst2[i-1]
        current_element = element
        h = and_merge(previous_element,current_element)
        print(h)
        print(len(h))
        new_lst.append(h)
    nested_list = list(more_itertools.collapse(new_lst))
    #print(nested_list)
    dupes = [x for n, x in enumerate(nested_list) if x in nested_list[:n]]
    print(dupes)
    k = list(dupes)
    print('total doc match', len(k))
    for i in k:
        display_summary(i, show_abstract = True)
        
c = 'basidiomycete Heterobasidion'
d = and_query(c)  

[8292264, 11314963, 11373666, 12742063, 15119351, 15465393, 18637961, 22319614, 23517685, 23645035, 23864721, 24286289, 27317690, 28801666, 30727562, 30832017]
16
[8292264, 11314963, 11373666, 12742063, 15119351, 15465393, 18637961, 22319614, 23517685, 23645035, 23864721, 24286289, 27317690, 28801666, 30727562, 30832017]
16
[8292264, 11314963, 11373666, 12742063, 15119351, 15465393, 18637961, 22319614, 23517685, 23645035, 23864721, 24286289, 27317690, 28801666, 30727562, 30832017]
total doc match 16


In [43]:
import more_itertools
#retrieving a query with three words!
def and_query(a_query):
    elem = list(preprocess(tokenize(a_query)))
    print(elem)
    first_elem = elem[0]
    first_list = inverted_index[first_elem]
    new_list = []
    for i in elem:
        docids = inverted_index[i]
        c = and_merge(first_list,docids)
        #print(c)
        print(len(c))
        new_list.append(c)
    nested_list = list(more_itertools.collapse(new_list))
    dupes = [x for n, x in enumerate(nested_list) if x in nested_list[:n]]
    k = list(dupes)
    print(k)
    #w = list(new_list)
    #print(new_list)
    #print(len(new_list))
    print('total doc match',len(k))
    #for i in k:
        #display_summary(i, show_abstract = True)
        
    
    
c = 'basidiomycete Heterobasidion annosum'
d = and_query(c)  

['basidiomycete', 'heterobasidion', 'annosum']
819
16
13
[8292264, 11314963, 11373666, 12742063, 15119351, 15465393, 18637961, 22319614, 23517685, 23645035, 23864721, 24286289, 27317690, 28801666, 30727562, 30832017, 8292264, 11314963, 11373666, 12742063, 15119351, 22319614, 23645035, 23864721, 24286289, 27317690, 28801666, 30727562, 30832017]
total doc match 29


### Task 4

Construct another function called `or_query` that works in the same way as `and_query` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query, using the `or_merge` function you defined.

Demonstrate the working of this function also with an example (again, choose one that leads to fewer than 100 hits).

In [15]:
def or_query(a_query):
    merge_list = []
    elem = preprocess(tokenize(a_query))
    merge_list = inverted_index[elem[0]]
    for word in elem[1:]:
        merge_list = or_merge(merge_list, inverted_index[word])
    
    return merge_list
        
c = 'basidiomycete Heterobasidion annosum'
d = or_query(c)  
for i in d:
        display_summary(i, show_abstract = True)
print(len(d))

883


### Task 5

Why does `and_query('spore infection reservoirs')` not return our example paper 28425998, even though it mentions spores, infection, and reservoirs in the abstract? (You do not have to implement anything to fix this yet!)

In [13]:
#retrieving a query with two words!
def and_query(a_query):
    merge_list = []
    elem = preprocess(tokenize(a_query))
    merge_list = inverted_index[elem[0]]
    for word in elem[1:]:
        merge_list = and_merge(merge_list, inverted_index[word])
    
    return merge_list
        
c = 'spore infection reservoirs'
d = and_query(c)  
for i in d:
        display_summary(i, show_abstract = True)
print(len(d))

0
